# Generate “Active Knowledge” Drills

## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [37]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [38]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`


import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [39]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source
import scala.util.Random


import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}


import java.io.File

import java.io.PrintWriter


import scala.io.Source

import scala.util.Random


## Useful Functions

Load a file

In [53]:
def loadFile( fp: String ): Vector[String] = {
    Source.fromFile(fp).getLines.toVector
}


defined function loadFile

Save a string to a names file:

In [54]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

defined function saveString

Like `.split`, but preserving the character we split on:

In [55]:
def splitWithSplitter(text: String, puncs: String): Vector[String] = {
	//val regexWithSplitter = s"((?<=${puncs})|(?=${puncs}))"
    val regexWithSplitter = s"((?<=${puncs}))"
	text.split(regexWithSplitter).toVector.filter(_.size > 0)
}

defined function splitWithSplitter

Pretty Print Things:

In [56]:
def showMe(v:Any):Unit = {
  v match {
    case _:StringHistogram => {
        for ( h <- v.asInstanceOf[StringHistogram].histogram ) {
            println(s"${h.count}\t${h.s}")
        }
    }
  	case _:Corpus => {
  		for ( n <- v.asInstanceOf[Corpus].nodes) {
  			println(s"${n.urn.passageComponent}\t\t${n.text}")
  		}	
  	}
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

defined function showMe

Shuffle a Vector of Strings

In [57]:
def shuffle(v: Vector[String]): Vector[String] = {
    val rr: Vector[Float] = v.map( i => {
        scala.util.Random.nextFloat
    })
    val zipped: Vector[(String,Float)] = v.zip(rr)
    zipped.sortBy( m => m._2).map(_._1)
}

defined function shuffle

## GIFT-Generation

We load two files, a template and some morphological data. These get zipped together. This saves a lot of typing, since *all* "adjectives of three terminations" are going to have the same list of identifications ("m/n/s, f/n/s, …"), etc. 

We define our inputs (at the bottom), specify how many items we want in our output, and the name of the output file.

Finally, we invoke the command `makeActiveParadigmQuiz(…)`, which generates a set of multiple choice questions, in `.gift` format, ready for Moodle. 

For the record… when setting up a Moodle “quiz” that you mean to be a drill , select “Adaptive Mode (no penalties)” under “Question Behavor”. This lets your students click an answer, click “check”, see if it is right, and amend their answer if it was wrong. So they should *never* get to the end of a drill with a score lower than 100%.

What follows, below, are some functions that do this work. But we start with some basic Classes: `ParsedForm`,`IndexedParsedForm`, and `GradedParsedForm`.

## Classes

In [58]:
case class ParsedForm( parse: String, form: String)

defined class ParsedForm

In [59]:
case class IndexedParsedForm( index: Int, pform: ParsedForm)

defined class IndexedParsedForm

In [60]:
case class GradedForm( grade: Int, form: String)

defined class GradedForm

## Pick a Card

This function is for selecting the *wrong* answers for the multiple-choice drill. We have a *correct* answer, but we need a certain number of *wrong* answers.

The challenge is to avoid picking "bad" wrong answers. For example, we don't want, through random selection, to pick multiple instances of the original *correct* answer.

It is based on the metaphor of a deck of cards, from which we pick a certain number. The "deck" is a list of `IndexedParsedForm`s. We parameterize several things:

- How many to pick
- An initial one _not_ to pick

This function uses tail-recursion.

    (Before we do this, though, let's make a method to re-index a Vector[IndexedParsedForm]`.

In [61]:
def reindexParsedForms( idff: Vector[IndexedParsedForm]): Vector[IndexedParsedForm] = {
    val noIndices: Vector[ParsedForm] = idff.map(_.pform)
    noIndices.zipWithIndex.map( ni => {
        val idx: Int = ni._2
        val pf: ParsedForm = ni._1
        IndexedParsedForm(idx, pf)
    })
}

defined function reindexParsedForms

In [62]:
def pickACard( 
    cards: Vector[IndexedParsedForm], 
    howMany: Int, 
    joker: Option[IndexedParsedForm] ): Vector[IndexedParsedForm] = {

    /*
        After the code below this new `def` is executed, this function
        will be invoked until we've picked the required number of "cards".
    */
    def pickRecurse( 
        ourHand: Vector[IndexedParsedForm], 
        whatsLeft: Vector[IndexedParsedForm]
    ): Vector[IndexedParsedForm] = {
        
        if (ourHand.size == howMany) reindexParsedForms(ourHand)
        else {
            val limit = whatsLeft.size
            // Get a random next item!
            val r = scala.util.Random
            val pickedIndex: Int = r.nextInt(limit)
            // We have confidence in our re-indexing!
            val pickedCard: IndexedParsedForm = whatsLeft.filter( wl => {
                wl.index == pickedIndex
            }).head
            
            // Add picked card to our "hand"
            val newHand: Vector[IndexedParsedForm] = ourHand :+ pickedCard
            
            // We remove the picked card from the deck
            val newDeck: Vector[IndexedParsedForm] = {
                val withCardRemoved = whatsLeft.filter( _ != pickedCard )
                // And so as not to bore the user with repeated forms…
                val withCardsFormsRemoved = withCardRemoved.filter( c => {
                    c.pform.form != pickedCard.pform.form
                })
                // and re-index…
                reindexParsedForms(withCardsFormsRemoved)
            }
            // recurse!
            pickRecurse( newHand, newDeck)
        }
        
    }
    
    // remove the joker from the deck
    val noJokers: Vector[IndexedParsedForm] = {
        joker match {
        case Some(j) => {
            // We remove the initial joker…
            val origJokerGone: Vector[IndexedParsedForm] = cards.filter( c => {
                c.index != j.index
            })
            /* Because this is an _active_ drill, we want to eliminate 
               all FORMS that match that of the Joker,
               otherwise, students may have to click on 
               three instances of "τῶν", to answer "definitive article: masc. gen. pl."
            */
            val matchingFormsGone: Vector[IndexedParsedForm] = origJokerGone.filter( c => {
                c.pform.form != j.pform.form
            })
            // We reindex!
            reindexParsedForms(matchingFormsGone)
        }
        case None => cards
        }
    }
    
    /* Now that we've gotten rid of the "joker", we can set up the recurse… */
    
    // emptyVec will be the starting value for `ourHand`
    val emptyVec: Vector[IndexedParsedForm] = Vector[IndexedParsedForm]()
	pickRecurse( emptyVec, noJokers )
}

defined function pickACard

## Make Question

This function assembles everything needed to create a drill question. 

What is needed is:

- A correct answer: a `ParsedForm`
- A Vector of other answers: a `Vector[ParsedForm]`

But it is more complicated than that. Obviously, our correct answer is correct. But it may be that some of our other answers are also correct. E.g. "The feminine genitive singular of 'who?'" might be **τίνος**, but it might be **τοῦ**. If **τοῦ** shows up, randomly, as an "other" answer, we need, not only to give the student credit for picking it, to *insist* that the user pick both **τοῦ** and **τίνος**.

This is where `GradedForm` comes in.

In [63]:
def makeQ( correct: IndexedParsedForm, deck: Vector[IndexedParsedForm], choices: Int = 5): Vector[GradedForm] = {
   
    val otherAnswers: Vector[IndexedParsedForm] = pickACard( deck, choices, Some(correct))
    
    // We might end up with more than one correct answer…
    val numberCorrect: Int = {
        otherAnswers.filter( a => {
            a.pform.parse == correct.pform.parse
        }).size + 1
    }
    
    // Make GradedForms
    val allAnswers: Vector[IndexedParsedForm] = correct +: otherAnswers
    val gradedAnswers: Vector[GradedForm] = allAnswers.map( aa => {
        val form = aa.pform.form
        // Is this one correct?
        val isCorrect: Boolean = ( aa.pform.parse == correct.pform.parse ) 
        val grade: Int = {
            if (isCorrect) 100 / numberCorrect
            else -100
        }
        GradedForm(grade, form)
    })
    gradedAnswers
}

defined function makeQ

Turn question-data into a GIFT string.

In [64]:
def makeGiftQuestion(data: Vector[GradedForm], head: String): String = {
    ""
}

defined function makeGiftQuestion

Make the actual drill!

In [68]:
def shuffleParsedForms( forms: Vector[ParsedForm]): Vector[ParsedForm] = {
    val rr: Vector[Float] = forms.map( i => {
        scala.util.Random.nextFloat
    })
    val zipped: Vector[(ParsedForm,Float)] = forms.zip(rr)
    zipped.sortBy( m => m._2).map(_._1)
}

def makeGiftDrill( 
    dataPath: String, 
    template: String,
    morph: String,
    outputPath: String,
    outputName: String,
    howMany: Int,
    howManyChoices: Int = 5
): Unit = {
    
    val forms: Vector[String] = {
        loadFile(s"${dataPath}${morph}")
    }

    val parsings: Vector[String] = {
        loadFile(s"${dataPath}${template}")
    }
    
    val formName: String = forms.head

    val parsedForms: Vector[ParsedForm] = parsings.tail.zip(forms.tail).map( t => {
        ParsedForm( t._1, t._2)
    })

    /* 
        Here is where howMany comes in. There are several cases to consider
        - howMany is 0 (= make one for each form in the data)
        - howMany is less than howManyChoices (won't do…)
        - howMany is less than the total number of forms
        - howMany is greater than the total number of forms
    */
    
    val useThese: Vector[ParsedForm] = {
        if (howMany == 0) shuffleParsedForms(parsedForms)
        else {
            if (howMany <= parsedForms.size) {
                if (howMany < howManyChoices) shuffleParsedForms(parsedForms).take(howManyChoices)
                else shuffleParsedForms(parsedForms).take(howMany)
            } else {
                val howManyCopies: Int = (howMany / parsedForms.size) + 1
                val longList: Vector[ParsedForm] = {
                    (1 to howManyCopies).toVector.map(c => {
                        parsedForms
                    }).flatten
                }
                shuffleParsedForms(longList).take(howMany)
            }
        }
    }
    
    

    
}

defined function shuffleParsedForms
defined function makeGiftDrill

## Do it!

Set up your data:

In [52]:
val morphDataDir: String = "morphology/"
val giftDir: String = "gifts/"

val morphData: String = "def_article.txt"
val template: String = "template_adjective_3.txt"
val fileName: String = "active_definite_article.gift"

val howMany: Int = 50 // How many questions to make, set to 0 to make one item for each form in the data.
val howManyChoices: Int = 5 // How many multiple-choices to offer

makeGiftDrill( 
    morphDataDir,
    template,
    morphData,
    giftDir,
    fileName,
    howMany
)

morphDataDir: String = "morphology/"
giftDir: String = "gifts/"
morphData: String = "def_article.txt"
template: String = "template_adjective_3.txt"
fileName: String = "active_definite_article.gift"
howMany: Int = 50